### Import Libraries

In [5]:
import os, cv2
from IPython.display import display, clear_output
from PIL import Image
import numpy as np

### Define the paths to Haar Cascade XML files for face detection in OpenCV

In [8]:
casc_path = cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml"
face_cascade = cv2.CascadeClassifier(casc_path)
if face_cascade.empty():
    raise FileNotFoundError(f"Failed to load cascade at: {casc_path}")

### Read the mask filter

In [9]:
mask_path = r"mini-mouse.png"  #use another image as proof of concept
mask_img = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  #allow alpha if present
if mask_img is None:
    raise FileNotFoundError(f"Mask image not found at: {mask_path}")

In [10]:
if mask_img.shape[2] == 4:
    mask_bgr = mask_img[:, :, :3]
    mask_alpha = mask_img[:, :, 3]
else:
    mask_bgr = mask_img
    mask_alpha = cv2.cvtColor(mask_bgr, cv2.COLOR_BGR2GRAY)

### Video Capture

In [ ]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not access the webcam (index 0).")

try:
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.2, 5, minSize=(100, 100))

        for (x, y, w, h) in faces:
            m_resized = cv2.resize(mask_bgr, (w, h), interpolation=cv2.INTER_LINEAR)
            a_resized = cv2.resize(mask_alpha, (w, h), interpolation=cv2.INTER_LINEAR)

            #make binary foreground mask from alpha/intensity
            _, fg_mask = cv2.threshold(a_resized, 25, 255, cv2.THRESH_BINARY)
            bg_mask = cv2.bitwise_not(fg_mask)

            roi = frame[y:y+h, x:x+w]
            bg = cv2.bitwise_and(roi, roi, mask=bg_mask)
            fg = cv2.bitwise_and(m_resized, m_resized, mask=fg_mask)
            frame[y:y+h, x:x+w] = cv2.add(bg, fg)

        #show in notebook
        clear_output(wait=True)
        display(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
except KeyboardInterrupt:
    pass
finally:
    cap.release()